In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()
filepath = "emp_data.csv"
df = spark.read.csv(filepath , header=True,inferSchema=True)
df.printSchema()
df.show()

root
 |-- EmpID: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- ExitDate: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Supervisor: string (nullable = true)
 |-- BusinessUnit: string (nullable = true)
 |-- EmployeeStatus: string (nullable = true)
 |-- EmployeeType: string (nullable = true)
 |-- PayZone: string (nullable = true)
 |-- EmployeeClassificationType: string (nullable = true)
 |-- TerminationType: string (nullable = true)
 |-- DepartmentType: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- State: string (nullable = true)
 |-- JobFunctionDescription: string (nullable = true)
 |-- GenderCode: string (nullable = true)
 |-- LocationCode: integer (nullable = true)
 |-- RaceDesc: string (nullable = true)
 |-- MaritalDesc: string (nullable = true)
 |-- Performance Score: string (nullable = true)
 |-- Cu

In [4]:
#to check null values for each 
df.select([count(when(isnull(c) | isnan(c) , c) ).alias(c) for c in df.columns]).show()

+-----+---------+--------+---------+--------+-----+----------+------------+--------------+------------+-------+--------------------------+---------------+--------------+--------+---+-----+----------------------+----------+------------+--------+-----------+-----------------+-----------------------+-------------+
|EmpID|FirstName|LastName|StartDate|ExitDate|Title|Supervisor|BusinessUnit|EmployeeStatus|EmployeeType|PayZone|EmployeeClassificationType|TerminationType|DepartmentType|Division|DOB|State|JobFunctionDescription|GenderCode|LocationCode|RaceDesc|MaritalDesc|Performance Score|Current Employee Rating|FormattedDate|
+-----+---------+--------+---------+--------+-----+----------+------------+--------------+------------+-------+--------------------------+---------------+--------------+--------+---+-----+----------------------+----------+------------+--------+-----------+-----------------+-----------------------+-------------+
|    0|        0|       0|        0|     263|    0|         0

In [5]:
df.printSchema()

root
 |-- EmpID: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- ExitDate: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Supervisor: string (nullable = true)
 |-- BusinessUnit: string (nullable = true)
 |-- EmployeeStatus: string (nullable = true)
 |-- EmployeeType: string (nullable = true)
 |-- PayZone: string (nullable = true)
 |-- EmployeeClassificationType: string (nullable = true)
 |-- TerminationType: string (nullable = true)
 |-- DepartmentType: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- State: string (nullable = true)
 |-- JobFunctionDescription: string (nullable = true)
 |-- GenderCode: string (nullable = true)
 |-- LocationCode: integer (nullable = true)
 |-- RaceDesc: string (nullable = true)
 |-- MaritalDesc: string (nullable = true)
 |-- Performance Score: string (nullable = true)
 |-- Cu

In [8]:
#1
df = df.withColumn("LastName",when(isnull(col("LastName") ), "Unknown").otherwise(col("LastName")))
df = df.dropna(subset = ["EmpID", "StartDate"])

In [11]:
df.count()

552

In [15]:
#2 remove outliers
# Cap current empliyee rating between 1 and 5
df = df.withColumn("Current Employee Rating", when(col("Current Employee Rating")<1,1).when(col("Current Employee Rating")>4,4).otherwise(col("Current Employee Rating")))

df.select("LocationCode").distinct().show()

+------------+
|LocationCode|
+------------+
|        2122|
|       97413|
|       80424|
|       56687|
|       75321|
|       49449|
|        1460|
|       78046|
|        9454|
|       44553|
|        8779|
|       29811|
|       57754|
|       65321|
|       39859|
|       74388|
|       31207|
|       34011|
|       49914|
|       60107|
+------------+
only showing top 20 rows



In [16]:
#checking values of cer
df.select("Current Employee Rating").distinct().show()

+-----------------------+
|Current Employee Rating|
+-----------------------+
|                      1|
|                      3|
|                      4|
|                      2|
+-----------------------+



In [17]:
#3
df=df.dropDuplicates()

In [18]:
df.count()

552

In [19]:
#4 count the number of employees for each department for every designation
ans = df.groupBy(trim("DepartmentType"),trim("Title")).count()
ans.show()

+--------------------+--------------------+-----+
|trim(DepartmentType)|         trim(Title)|count|
+--------------------+--------------------+-----+
|               IT/IS|Principal Data Ar...|    8|
|               IT/IS|        Data Analyst|   11|
|          Production|Production Techni...|   61|
|               IT/IS|Sr. Network Engineer|   30|
|          Production|    Network Engineer|    1|
|    Executive Office|    Network Engineer|   18|
|               IT/IS|    Network Engineer|   22|
|          Production|Production Techni...|    1|
|          Production|Production Techni...|  190|
|               IT/IS|      Data Architect|    2|
|               IT/IS|  Area Sales Manager|    7|
|               Sales|  Area Sales Manager|  119|
|               IT/IS|Enterprise Architect|    5|
|               IT/IS|          IT Support|   44|
|               Sales| Area Sales Manager?|    1|
|               Sales| Area Sales Manager.|    1|
|               IT/IS|Software Engineer...|    1|


In [21]:
#5 for each department find the employee with thte highest performance score
ans = df.groupBy(trim("DepartmentType").alias("Dept")).agg(max("Current Employee Rating").alias("MaxPerformanceScore"))
ans.show()

+--------------------+-------------------+
|                Dept|MaxPerformanceScore|
+--------------------+-------------------+
|               Sales|                  4|
|          Production|                  4|
|    Executive Office|                  3|
|Software Engineering|                  3|
|               IT/IS|                  4|
+--------------------+-------------------+



In [ ]:
pandas_df = df.toPandas()
pandas_df.to_csv(output_path,index=False)